## 生成数据集

这里先固定好整个**SFC**的结构，也就是一共几类**VNF**，以及每一类**SFC**的实例个数，以及**QOS**属性的数量和前向属性的个数**K**。

可以变动的参数有：
- 每一个**VNF**当前的**QOS**取值
- 每一个属性的权重

--------------------------

那么生成的样本的**输入属性**就是:所有**VNF**当前的**QOS**，以及每一个**属性**的权重组成的一个向量

样本的**标签**为每一类VNF的选择。

--------------------------

<div align="center"><img src="sfc_show.png" style="zoom:65%" title="SFC示意图" /></div>

In [ ]:
import numpy as np
import sfc_model as sfc

### 确定到网络的结构

```python
__init__(self, num_i = [2, 3], scope_L = [[1, 1000],[1, 100]], K = 1)
```

`num_i`为每一类**VNF**的数量

`scope_L` 为每一个**QOS**属性的取值范围

`K` 为前向属性的数量

In [ ]:
num_i = [3,4,5]

scope_L = [[1, 1000],[1, 100],[1,100],[1,1000]]

K = 2

my_model = sfc.sfc_model(num_i = num_i, scope_L = scope_L, K = K)

### 随机QOS请求

首先确定出**QOS**请求的**范围**

注意到应该有**QOS**不能被**SFC**所满足

那么这里将它的范围上限定义为属性的上限乘上**VNF**的种类数，下限就为属性的下限。


In [ ]:
def random_qos( N = 3, scope_L = [[1, 1000],[1, 100],[1,100],[1,1000]], K = 2 ):
    L = len(scope_L)
    qos = np.zeros(L)
    for i in range(L):
        qos[i] = np.random.rand(1)[0] * (scope_L[i][1] * N - scope_L[i][0]) + scope_L[i][0]
#         if i < K:
#             qos[i] = np.random.rand(1)[0] * (scope_L[i][1] * N - scope_L[i][0]) + scope_L[i][0]
#         else:
#             qos[i] = np.random.rand(1)[0] * (scope_L[i][1] * N - scope_L[i][0]) + scope_L[i][0]
    
    return qos

### 生成样本写入文件

先生成一百万条样本试一下

写成**CSV**文件

In [ ]:
## 把 path 变成一个分类问题

N = 3
back_muti = np.zeros(N)
category = 1
for i in range(N):
    back_muti[i] = 1
    category = category * num_i[i]
    for j in range(i+1, N):
        back_muti[i] = back_muti[i] * num_i[j]

def get_category(path):
    if path is None:
        return category
    
    cate = 0
    for i in range(N):
        cate = cate + path[i] * back_muti[i]
    
    return int(cate)

In [ ]:
T = 100 
N = 3
num_i = [3,4,5]
samples = np.zeros((T, len(scope_L)*np.sum(num_i) + len(scope_L)*2))
# labels = np.zeros((T, len(num_i) + np.sum(num_i)))
labels = np.zeros((T, category+1))
for t in range(T):
    qos = random_qos()
    my_model.construct()
    path, _ = my_model.serveQOS(random_qos())
    sample = qos
    sample = np.hstack((sample, my_model.W))
    sample = np.hstack((sample, my_model.qoses.reshape(-1)))
    label = np.zeros(category+1)
    idx = get_category(path)
    label[idx] = 1
#     for n in range(N):
#         one_hot = np.zeros(num_i[n] + 1)
#         if path is None:
#             one_hot[num_i[n]] = 1
#         else:
#             one_hot[path[n]] = 1
            
#         if label is None:
#             label = one_hot
#         else:
#             label = np.hstack((label, one_hot))            
    samples[t] = sample.reshape(1,-1)
    labels[t] = label.reshape(1,-1)
    
    if t % 1000 == 0:
        print(t)

In [ ]:
np.savetxt('data/X_train.csv', samples, delimiter = ',')
np.savetxt('data/Y_train.csv', labels, delimiter = ',')